In [1]:
# Store longitude, latitude and street crossing name of each public library location.
class XPoint(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __str__(self):
        return "P(%g_%g)" % (self.x, self.y)

class NamedPoint(XPoint):
    def __init__(self, name, x, y):
        XPoint.__init__(self, x, y)
        self.name = name
    def __str__(self):
        return self.name

In [2]:
# Simple distance computation between 2 locations.
from geopy.distance import great_circle

def get_distance(p1, p2):
    return great_circle((p1.y, p1.x), (p2.y, p2.x)).miles


p1 = NamedPoint('t1',30,30)
p2 = NamedPoint('t2',-30,70)

In [3]:
def build_libraries_from_url(url):
    import requests
    import json

    r = requests.get(url)
    myjson = json.loads(r.text, parse_constant='utf-8')
    myjson = myjson['data']

    libraries = []
    k = 1
    for location in myjson:
        uname = location[8]
        name = str(uname)
        
        data  = location[16]
        latitude = float(data[1])
        longitude = float(data[2])

        name = "Point_%s_%d" % (name, k)
        if latitude and longitude:
            cp = NamedPoint(name, longitude, latitude)
            libraries.append(cp)
            k += 1
    return libraries

libraries = build_libraries_from_url('https://data.cityofchicago.org/api/views/x8fc-8rcq/rows.json?accessType=DOWNLOAD')
                
print("There are %d public libraries in Chicago" % (len(libraries)))

There are 81 public libraries in Chicago


In [4]:
nb_shops = 8
print("We would like to open %d coffee shops" % nb_shops)

We would like to open 8 coffee shops


In [5]:
import folium
map_osm = folium.Map(location=[41.878, -87.629], zoom_start=11)
for library in libraries:
    lt = library.y
    lg = library.x
    folium.Marker([lt, lg]).add_to(map_osm)
map_osm

/home/pedro/anaconda2/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


In [6]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python is present, version is 2.7.16
* docplex is present, version is (2, 9, 141)
* CPLEX wrapper is present, version is 12.9.0.0, located at: /home/pedro/anaconda2/lib/python2.7/site-packages


In [7]:
from docplex.mp.model import Model

mdl = Model("coffee shops")

In [8]:
BIGNUM = 999999999

# assumimos que pode existir um café em cada livraria
coffeeshop_locations = set(libraries)

# Decision vars
# Binary vars indicating which coffee shop locations will be actually selected
coffeeshop_vars = mdl.binary_var_dict(coffeeshop_locations, name="is_coffeeshop")

#
# Binary vars representing the "assigned" libraries for each coffee shop
link_vars = mdl.binary_var_matrix(coffeeshop_locations, libraries, "link")

In [9]:
for c in coffeeshop_locations:
    for l in libraries:
        if get_distance(c, l) >= BIGNUM:
            mdl.add_constraint(link_vars[c, l] == 0, "ct_forbid_{0!s}_{1!s}".format(c_loc, b))

In [10]:
b = mdl.add_constraints(link_vars[c_loc, b] <= coffeeshop_vars[c_loc]
                   for b in libraries
                   for c_loc in coffeeshop_locations)
mdl.print_information()

Model: coffee shops
 - number of variables: 6642
   - binary=6642, integer=0, continuous=0
 - number of constraints: 6561
   - linear=6561
 - parameters: defaults


In [11]:
mdl.add_constraints(mdl.sum(link_vars[c_loc, b] for c_loc in coffeeshop_locations) == 1
                   for b in libraries)
mdl.print_information()

Model: coffee shops
 - number of variables: 6642
   - binary=6642, integer=0, continuous=0
 - number of constraints: 6642
   - linear=6642
 - parameters: defaults


In [12]:
# Total nb of open coffee shops
mdl.add_constraint(mdl.sum(coffeeshop_vars[c_loc] for c_loc in coffeeshop_locations) == nb_shops)

# Print model information
mdl.print_information()

Model: coffee shops
 - number of variables: 6642
   - binary=6642, integer=0, continuous=0
 - number of constraints: 6643
   - linear=6643
 - parameters: defaults


In [13]:
# Minimize total distance from points to hubs
total_distance = mdl.sum(link_vars[c_loc, b] * get_distance(c_loc, b) for c_loc in coffeeshop_locations for b in libraries)
mdl.minimize(total_distance)

In [14]:
print("# coffee shops locations = %d" % len(coffeeshop_locations))
print("# coffee shops           = %d" % nb_shops)

# coffee shops locations = 81
# coffee shops           = 8


In [15]:
assert mdl.solve(log_output = True), "!!! Solve of the model fails"

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced MIP has 6643 rows, 6642 columns, and 19764 nonzeros.
Reduced MIP has 6642 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.07 sec. (28.16 ticks)
Found incumbent of value 628.029462 after 0.17 sec. (65.30 ticks)
Probing time = 0.10 sec. (4.22 ticks)
Tried aggregator 1 time.
Reduced MIP has 6643 rows, 6642 columns, and 19764 nonzeros.
Reduced MIP has 6642 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (30.75 ticks)
Probing time = 0.03 sec. (4.22 ticks)
Clique table members: 6642.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.18 sec. (39.99 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                          628.0295       

In [16]:
total_distance = mdl.objective_value
open_coffeeshops = [c_loc for c_loc in coffeeshop_locations if coffeeshop_vars[c_loc].solution_value == 1]
not_coffeeshops = [c_loc for c_loc in coffeeshop_locations if c_loc not in open_coffeeshops]
edges = [(c_loc, b) for b in libraries for c_loc in coffeeshop_locations if int(link_vars[c_loc, b]) == 1]

print("Total distance = %g" % total_distance)
print("# coffee shops  = {0}".format(len(open_coffeeshops)))
for c in open_coffeeshops:
    print("new coffee shop: {0!s}".format(c))

Total distance = 159.369
# coffee shops  = 8
new coffee shop: Point_Canaryville_15
new coffee shop: Point_Legler Regional_40
new coffee shop: Point_West Loop_76
new coffee shop: Point_Beverly_8
new coffee shop: Point_Jeffery Manor_37
new coffee shop: Point_Austin-Irving_5
new coffee shop: Point_Sulzer Regional Library_66
new coffee shop: Point_West Lawn_75


In [17]:
import folium
map_osm = folium.Map(location=[41.878, -87.629], zoom_start=11)
for coffeeshop in open_coffeeshops:
    lt = coffeeshop.y
    lg = coffeeshop.x
    folium.Marker([lt, lg], icon=folium.Icon(color='red',icon='info-sign')).add_to(map_osm)

for b in libraries:
    if b not in open_coffeeshops:
        lt = b.y
        lg = b.x
        folium.Marker([lt, lg]).add_to(map_osm)


for (c, b) in edges:
    coordinates = [[c.y, c.x], [b.y, b.x]]
    map_osm.add_children(folium.PolyLine(coordinates, color='#FF0000', weight=5))

map_osm

/home/pedro/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
